# Get MF-DFA on Surrogate Data
#### Date: 9/07/2022, JX
> Make IAAFT and get surrogate data on /mnt/cube/j8xing/starling_rhythm_mfdfa/data/starling_zenodo_2019_amp_envs_MF.pickle

In [1]:
from starling_rhythm.utils.paths import PROCESSED_DIR

## Import dataframe

In [2]:
import pandas as pd

In [3]:
SAVE_PATH = PROCESSED_DIR / 'starling_zenodo_2019_amp_envs_MF.pickle'
ae_df = pd.read_pickle(SAVE_PATH)

## Pick 50 songs from each bird

In [4]:
import numpy as np

In [5]:
selected_ae_df = []
for bID in np.unique(ae_df['bID'].values):
    selected_ae_df.append(ae_df[ae_df['bID'] == bID].sample(10))

selected_ae_df = pd.concat(selected_ae_df)

In [6]:
selected_ae_df

,bID,wav_file,amp_env,h_expo,MF_range
5838,B335,2008-06-08_18-34-00-000000.wav,"[0.00014099797408562154, 0.004352819174528122,...","[1.2976591156234654, 1.2148145479568389, 1.161...",0.283754
5477,B335,2008-05-29_03-30-00-000000.wav,"[8.393513417104259e-05, 0.0034067369997501373,...","[1.2793530972682385, 1.1901398253641458, 1.137...",0.279412
5540,B335,2008-06-09_09-38-00-000000.wav,"[0.00032850034767761827, 0.003251151880249381,...","[1.2581912026325481, 1.1709291887417856, 1.109...",0.342261
5501,B335,2008-05-29_05-47-00-000000.wav,"[0.00023563225113321096, 0.012154174037277699,...","[1.271233499167112, 1.1611914481194776, 1.0871...",0.364055
5968,B335,2008-06-15_22-05-00-000000.wav,"[0.0029380263295024633, 0.047291193157434464, ...","[1.2881788974714221, 1.2204863232790821, 1.173...",0.260816
...,...,...,...,...,...
4903,b1160,2017-11-17_13-04-28-200000.wav,"[0.000305128691252321, 0.005687038879841566, 0...","[1.3169211098678355, 1.2166390263904543, 1.148...",0.330126
4950,b1160,2017-11-17_08-55-07-410000.wav,"[0.0005536586977541447, 0.012306184507906437, ...","[1.3103592342345496, 1.2083559005665407, 1.129...",0.385369
5004,b1160,2017-11-18_19-40-41-610000.wav,"[0.007926148362457752, 0.8716099262237549, 2.0...","[1.2870580621866805, 1.1862614770584456, 1.101...",0.429587
4983,b1160,2017-11-14_08-11-30-600000.wav,"[7.876505696913227e-05, 0.002603284316137433, ...","[1.2924508278944347, 1.180002264355716, 1.1034...",0.379017


## Generate IAAFT Surrogates
> 1. Store the amplitude spectrum (list of amplitudes corresponding to each frequency of a FT)
2. Randomized the series (build randomized phase spectrum)
3. Replace the amplitude spectrum in the randomized series with stored amplitude spectrum using inverse FT
4. Replace the rank-order value of iFT with ranked-ordered matches in the original series
5. Reiterate steps 2-4 until difference between empirical amplitude spectra and surrogate is less than 5 percent. 
## Retrieve Sampling Distribution of MF_range for each amplitude envelope
> Use surrogate count of 32 for $\alpha$ = 0.05, effect size = 0.5, power = 0.5 (See Kelty-Stephen et al. 2022)

In [7]:
import starling_rhythm.iaaft as iaaft
from starling_rhythm.mfdfa import hurst_expo
from joblib import Parallel, delayed
from timeit import timeit
from tqdm.autonotebook import tqdm

/home/AD/j8xing/anaconda3/envs/starling_rhythm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def SDoMF_iaaft(amp_env, ns = 32):
    '''
    For each amplitude envelope, find some number of its surrogates, and find MF_range for all surrogates
    '''
    
    ## calculate its surrogates
    surrogates = iaaft.surrogates(amp_env, ns = ns, maxiter = 1000, verbose = False)
    
    ## container for SDoMF_iaaft
    SDoMF = []
    
    ## find all the Hurst exponents of all the surrogates
    for surrogate in surrogates:
        ## Output Hs for each surrogate
        h_expos = hurst_expo(surrogate)
        SDoMF.append(max(h_expos) - min(h_expos))
            
    return SDoMF

In [9]:
n_jobs = 36 ## set up joblib parameters
verbose = 0

In [10]:
import psutil
psutil.Process().nice(18)

In [11]:
with Parallel(n_jobs = n_jobs, verbose = verbose) as parallel:
    mf_ranges = parallel(
        delayed(SDoMF_iaaft)(
            amp_env
        )
        for amp_env in tqdm(selected_ae_df.amp_env.values, desc = 'Running Surrogate Data')
    )

Running Surrogate Data: 100%|██████████| 140/140 [53:28<00:00, 22.91s/it]


maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

...

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb

In [12]:
selected_ae_df['mf_iaaft_range'] = mf_ranges

In [13]:
selected_ae_df

,bID,wav_file,amp_env,h_expo,MF_range,mf_iaaft_range
5838,B335,2008-06-08_18-34-00-000000.wav,"[0.00014099797408562154, 0.004352819174528122,...","[1.2976591156234654, 1.2148145479568389, 1.161...",0.283754,"[0.21218087028428378, 0.20811114782721596, 0.2..."
5477,B335,2008-05-29_03-30-00-000000.wav,"[8.393513417104259e-05, 0.0034067369997501373,...","[1.2793530972682385, 1.1901398253641458, 1.137...",0.279412,"[0.19699811587849836, 0.19925514443504144, 0.2..."
5540,B335,2008-06-09_09-38-00-000000.wav,"[0.00032850034767761827, 0.003251151880249381,...","[1.2581912026325481, 1.1709291887417856, 1.109...",0.342261,"[0.293409036254924, 0.31938674229728825, 0.259..."
5501,B335,2008-05-29_05-47-00-000000.wav,"[0.00023563225113321096, 0.012154174037277699,...","[1.271233499167112, 1.1611914481194776, 1.0871...",0.364055,"[0.2699477918473415, 0.28041314399793116, 0.30..."
5968,B335,2008-06-15_22-05-00-000000.wav,"[0.0029380263295024633, 0.047291193157434464, ...","[1.2881788974714221, 1.2204863232790821, 1.173...",0.260816,"[0.17158132725203745, 0.1739930085967445, 0.18..."
...,...,...,...,...,...,...
4903,b1160,2017-11-17_13-04-28-200000.wav,"[0.000305128691252321, 0.005687038879841566, 0...","[1.3169211098678355, 1.2166390263904543, 1.148...",0.330126,"[0.2501399304057381, 0.2276392425293874, 0.271..."
4950,b1160,2017-11-17_08-55-07-410000.wav,"[0.0005536586977541447, 0.012306184507906437, ...","[1.3103592342345496, 1.2083559005665407, 1.129...",0.385369,"[0.29698369830971927, 0.2924880500454896, 0.31..."
5004,b1160,2017-11-18_19-40-41-610000.wav,"[0.007926148362457752, 0.8716099262237549, 2.0...","[1.2870580621866805, 1.1862614770584456, 1.101...",0.429587,"[0.29047795310346436, 0.35556270483167585, 0.2..."
4983,b1160,2017-11-14_08-11-30-600000.wav,"[7.876505696913227e-05, 0.002603284316137433, ...","[1.2924508278944347, 1.180002264355716, 1.1034...",0.379017,"[0.26549814582728026, 0.2656934375028498, 0.32..."


## Pickle Dataframe

In [14]:
from starling_rhythm.utils.paths import PROCESSED_DIR, ensure_dir

In [15]:
SAVE_PATH = PROCESSED_DIR / 'starling_zenodo_2019_amp_envs_MF_IAAFT_selected.pickle'
ensure_dir(SAVE_PATH)

In [16]:
selected_ae_df.to_pickle(SAVE_PATH)

maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum number of iterations reached!
maximum numb